In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from extract_features_from_matterport import (
    MatterportFeature, MatterportDataset, Arguments, load_viewpoints, 
    init_detector,inference_detector,filter_panorama, cartesian_to_polar
)
import sys
from pathlib import Path
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as mcolors
from mmdet.datasets.lvis import LVISV1Dataset
from torch.nn import functional as F
import math
import matplotlib.colors as mcolors

classes = LVISV1Dataset.CLASSES

sys.argv= ['foo']
args = Arguments
args.matterport = Path('matterport-views.lmdb')
args.max_total_boxes = 360

In [ ]:
viewpoints = load_viewpoints(args)
dataset = MatterportDataset(viewpoints, args)

In [ ]:
feats, scan, viewpoint = dataset[2]

In [ ]:
scan, viewpoint

In [ ]:
device_id = 0
model = init_detector(str(args.config), str(args.checkpoint), device=f'cuda:{device_id}')

In [ ]:
all_boxes = []
all_features = []
all_probs = []
all_view_ids = []
all_labels = []

for view_id, im in zip(feats['view_ids'], feats['image_feat']):

    results = inference_detector(model, np.array(im))
    assert len(results['bbox']) == len(results['features'])

    all_boxes.append(results['bbox'])
    all_probs.append(results['cls_score'])
    all_features.append(results['features'])
    all_labels.append(results['labels'])
    num_bbox = results['bbox'].shape[0]
    all_view_ids += [view_id] * num_bbox

image_feat = torch.cat(all_features)
bbox = torch.cat(all_boxes)
probs = torch.cat(all_probs)
view_ids = torch.Tensor(all_view_ids)
labels = torch.cat(all_labels)

keep_ind = filter_panorama(
    bbox,
    probs,
    image_feat,
    view_ids,
    args.max_total_boxes, 
    feats['image_w'],
    feats['image_h'],
    feats['fov'],
)

In [ ]:
for view_id in range(36):
    mask = view_ids[keep_ind] == view_id
    if not mask.any():
        continue
    plt.imshow(feats['image_feat'][view_id])
    ax = plt.gca()

    for obj, vid, label, color in zip(bbox[keep_ind][mask], view_ids[keep_ind][mask], probs[keep_ind][mask], mcolors.TABLEAU_COLORS):
        if vid != view_id:
            continue
        h = obj[3] - obj[1]
        w = obj[2] - obj[0]
        pos = (obj[0], obj[1])
        rect = patches.Rectangle(
            (pos), w, h, linewidth=1, 
            label=classes[label.argmax()],
            edgecolor=color, 
            facecolor='none')
        ax.add_patch(rect)
    plt.axis('off')
    plt.legend()
    plt.show()